**Importing the required libraries**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split as tts
from sklearn.svm import SVR

**Reading the BTC.csv file which contains the data from April 29, 2013 to November 06, 2020. The data is scrapped from https://coinmarketcap.com/currencies/bitcoin/historical-data/?start=20130429&end=20201107 using Instant Data Scrapper(a free extension which can be used in a browser to easily scrap data from a website) https://chrome.google.com/webstore/detail/instant-data-scraper/ofaokhiedipichpaobibbnahnkdoiiah.**

In [2]:
print("Enter 1 for Bitcoin")
print("Enter 2 for Ethereum")
print("Enter 3 for Bitcoin Cash")
print("Enter 4 for Litecoin")
print("Enter 5 for Binance Coin")
print("Enter 6 for ChainLink")

choice = int(input("Enter Choice: "))

if choice == 1:
    df = pd.read_csv("Bitcoin.csv")
if choice == 2:
    df = pd.read_csv("Ethereum.csv")
if choice == 3:
    df = pd.read_csv("Bitcoin Cash.csv")
if choice == 4:
    df = pd.read_csv("Litecoin.csv")
if choice == 5:
    df = pd.read_csv("Binance Coin.csv")
if choice == 6:
    df = pd.read_csv("ChainLink.csv")

df.tail()

Enter 1 for Bitcoin
Enter 2 for Ethereum
Enter 3 for Bitcoin Cash
Enter 4 for Litecoin
Enter 5 for Binance Coin
Enter 6 for ChainLink
Enter Choice: 6


,cmc-table__cell,cmc-table__cell 2,cmc-table__cell 3,cmc-table__cell 4,cmc-table__cell 5,cmc-table__cell 6,cmc-table__cell 7
1147,"Sep 24, 2017",0.131110,0.190250,0.130275,0.170278,"1,025,740","59,597,300"
1148,"Sep 23, 2017",0.149562,0.157440,0.126297,0.131129,"1,186,670","45,895,150"
1149,"Sep 22, 2017",0.169826,0.181675,0.139869,0.150503,"1,423,220","52,676,050"
1150,"Sep 21, 2017",0.189132,0.207892,0.155292,0.169680,"2,126,270","59,388,000"
1151,"Sep 20, 2017",0.156494,0.189448,0.152489,0.189165,"2,718,640","66,207,750"


**Renaming the columns as given in the website https://coinmarketcap.com/currencies/bitcoin/historical-data/?start=20130429&end=20201107.**

In [3]:
df.columns = ['Date', 'Open', 'High', 'Low', 'Closing Price', 'Volume', 'Market Cap']
df.head()

,Date,Open,High,Low,Closing Price,Volume,Market Cap
0,"Nov 14, 2020",12.89,12.91,12.39,12.58,"997,198,275","4,936,577,002"
1,"Nov 13, 2020",12.42,12.99,12.34,12.89,"1,306,004,087","5,045,766,668"
2,"Nov 12, 2020",12.81,13.08,12.31,12.42,"1,400,960,180","4,863,810,509"
3,"Nov 11, 2020",13.00,13.39,12.79,12.81,"1,517,819,145","5,014,273,699"
4,"Nov 10, 2020",12.52,13.24,12.35,13.00,"1,724,813,657","5,090,980,020"


**Removing the unwanted columns**

In [4]:
df = df.filter(['Closing Price'])
df.head()

,Closing Price
0,12.58
1,12.89
2,12.42
3,12.81
4,13.00


**Reversing the Column**

In [5]:
df["Closing Price"] = df["Closing Price"].values[::-1]
df.head()

,Closing Price
0,0.189165
1,0.169680
2,0.150503
3,0.131129
4,0.170278


**If Bitcoin, Ethereum, Bitcoin Cash or any other cryptocurrency whose price is more then 999$. The "," needs to be removed as the data scrapped is in the form of string data-type and if "," isn't removed then it's not possible to convert it to float data-type without which no calculations can be made.**

In [6]:
if choice == 1 or choice == 2 or choice == 3: 
    df['Closing Price'] = df['Closing Price'].str.replace(r',', '')
    df['Closing Price'] = pd.to_numeric(df['Closing Price'])
else:
    pass

**Input the number of days for which the price is to be predicted**

In [7]:
prediction_days = int(input())

df['Prediction'] = df[['Closing Price']].shift(-prediction_days)

df.head()

10


,Closing Price,Prediction
0,0.189165,0.451364
1,0.169680,0.436647
2,0.150503,0.356767
3,0.131129,0.345535
4,0.170278,0.397361


**From the last the number of prediction_days is null as it is shifted up by prediction_days.** 

In [8]:
df.tail()

,Closing Price,Prediction
1147,13.00,NaN
1148,12.81,NaN
1149,12.42,NaN
1150,12.89,NaN
1151,12.58,NaN


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1152 entries, 0 to 1151
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Closing Price  1152 non-null   float64
 1   Prediction     1142 non-null   float64
dtypes: float64(2)
memory usage: 18.1 KB


In [10]:
df['Closing Price'] = pd.to_numeric(df['Closing Price'])

**Creating independent dataset**

**Converting the dataframe to numpy array and drop the prediction column.**

In [11]:

X = np.array(df.drop(['Prediction'],1))
X = X[:len(X)-prediction_days]
print(X)

[[ 0.189165]
 [ 0.16968 ]
 [ 0.150503]
 ...
 [10.8     ]
 [10.39    ]
 [10.47    ]]


**Creating dependent dataset**

In [12]:
Y = np.array(df["Prediction"])
Y = Y[:len(Y)-prediction_days]
print(Y)

[ 0.451364  0.436647  0.356767 ... 12.42     12.89     12.58    ]


**Split the data into 80% training and 20% testing**

In [13]:
x_train, x_test, y_train, y_test = tts(X, Y, test_size = 0.2)

In [14]:
prediction_array = np.array(df.drop(['Prediction'],1))
prediction_array = prediction_array[-prediction_days:]
print(prediction_array)

[[10.99]
 [12.12]
 [11.76]
 [12.66]
 [12.52]
 [13.  ]
 [12.81]
 [12.42]
 [12.89]
 [12.58]]


**Create and train the support vector machine (SVR - Suppport Vector Machine) using radial basis function(RBF)**

In [15]:
rbf = SVR(kernel = 'rbf', C = 1e3, gamma = 0.00001)
rbf.fit(x_train, y_train)

SVR(C=1000.0, gamma=1e-05)

**Testing the Model**

In [16]:
rbf_test = rbf.score(x_test, y_test)
print("Accuracy", rbf_test)

Accuracy 0.919614747241543


**Predicted Values**

In [17]:
prediction = rbf.predict(x_test)
print(prediction)

print()

print(y_test)

[ 4.50140825  0.49678312  2.44460004  0.32700776  0.49441929  4.37175628
  4.01261421  0.2565029   0.45506984  0.32032384  3.51355202  0.38185995
  2.80439023  0.36894056 18.63824808  0.44241279  0.52579361  0.30672146
  2.79439799  2.53456093  0.53291791  0.25200959  7.71584586  2.53456093
  4.53132503  3.31384601  0.54341245  4.44157142  0.21256081  3.85294613
  0.37600783  0.23177593  4.06250429  0.43410689  0.22655014  2.73444221
 12.34675164 10.69818542  3.0941167   0.57821456  0.57269005  2.71445607
  1.83463497  0.7051379   1.02843349  0.37674611  2.33463588  0.69511067
  0.51952163  3.79306284  2.86434135  2.70446283  4.10241421  0.59736083
  2.01466598  0.26438442  0.28589778  2.75442792  0.51851703 15.99194401
  0.45143581  0.23944525  0.38901829  0.32087483  0.2694878  10.72784018
  0.324897    0.22994651  2.22465863  0.60119464  0.70739309  0.38267436
  1.17423233  2.35463034  2.64450108  0.20029657  0.48858383  0.44552198
  0.25593585  0.34636904  1.87464482  0.28199267  2

**Print the prediction for next 'prediction_days' days.**

In [18]:
prediction = rbf.predict(prediction_array)
print(prediction)
print()
print(df.tail(prediction_days))

[10.97490749 12.09039014 11.73524218 12.62270603 12.48474541 12.95761427
 12.77048402 12.38618181 12.84928324 12.54387546]

      Closing Price  Prediction
1142          10.99         NaN
1143          12.12         NaN
1144          11.76         NaN
1145          12.66         NaN
1146          12.52         NaN
1147          13.00         NaN
1148          12.81         NaN
1149          12.42         NaN
1150          12.89         NaN
1151          12.58         NaN
